In [ ]:
# TODO

Note: the portfolios are cost-less (no transaction cost, no short-fees, no slippage, etc.). Also, we dont take tick-size into consideration, and assume we can purchase any fraction of a share.



In [2]:
import os
import yaml
from datetime import  timedelta

import string
import re 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from utils import flatten_list

In [3]:
%matplotlib inline

In [4]:
ROOT = os.path.dirname(os.getcwd())
PATH_SRC = os.path.join(ROOT, 'src')
PATH_DATA = os.path.join(ROOT, 'data')
PATH_DATA_PRICES = os.path.join(PATH_DATA, 'yfinance_prices')

In [5]:
min_week = pd.to_datetime('2020-09-03')
max_week = pd.to_datetime('2023-08-15')
dates = pd.date_range(min_week, max_week, freq='W') - timedelta(days=2)

In [6]:
# Load trades data
def date_parser(string_list):
    month_name = dict((k, v+1) for v, k in enumerate(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']))
    return [f"{x.split(' ')[2]} {int(month_name[x.split(' ')[4]]):02d} {int(x.split(' ')[3]):02d}" for x in string_list]

df_trades = pd.read_csv(
    os.path.join(PATH_DATA, 'CapitolTrades_raw.csv'),
    parse_dates=[
        'traded'
        ],
    date_parser=date_parser,
    usecols=[
        'politician',
        'trade_issuer',
        'ticker',
        'traded',
        'owner',
        'type',
        'size',
        'price'
        ],
    dtype={
        'owner': 'category',
        'politician': 'category',
        'type': 'category',
        },
)

# Check which firms have price data for
firms = [x.strip('.csv')for x in os.listdir(PATH_DATA_PRICES)]

# Format tickers to correspond with pricing data
df_trades.dropna(subset=['ticker'], inplace=True)
df_trades['ticker'] = df_trades['ticker'].apply(lambda x: x.strip(':US'))
df_trades['ticker'] = df_trades['ticker'].astype('category')

# Drop trades I don't have pricing data for
df_trades = df_trades[df_trades.ticker.isin(firms)]

# Drop small trades
df_trades = df_trades[df_trades['size'] != ' < 1K']

# Drop exchanges and recieves (because I don't know what they are)
df_trades = df_trades[df_trades['type'].isin([' buy', ' sell'])]
df_trades['type'] = df_trades['type'].cat.remove_unused_categories()

# Drop trades by children
df_trades = df_trades[df_trades['owner'] != ' Child']
df_trades['owner'] = df_trades['owner'].cat.remove_unused_categories()

# Round date to nearest friday (to get week closing price)
df_trades['week_date'] = pd.to_datetime(df_trades.traded.dt.to_period('W').dt.end_time.dt.date - timedelta(days=2))

df_trades.head(20)


,politician,trade_issuer,ticker,traded,owner,type,size,price,week_date
1,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Undisclosed,sell,15K–50K,78.29,2023-08-11
2,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Undisclosed,buy,15K–50K,77.89,2023-08-11
3,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Undisclosed,sell,15K–50K,78.29,2023-08-11
4,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Undisclosed,buy,15K–50K,77.89,2023-08-11
5,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Spouse,sell,15K–50K,78.29,2023-08-11
6,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Spouse,buy,15K–50K,77.89,2023-08-11
8,Bill Keating,PepsiCo Inc,PEP,2023-08-10,Undisclosed,buy,15K–50K,N/A,2023-08-11
11,Lois Frankel,Fluor Corp,FLR,2023-08-04,Undisclosed,sell,1K–15K,35.94,2023-08-04
12,Lois Frankel,Insperity Inc,NSP,2023-08-07,Undisclosed,sell,1K–15K,97.4,2023-08-11
13,Lois Frankel,Insperity Inc,NSP,2023-08-04,Undisclosed,sell,1K–15K,94.15,2023-08-04


In [7]:
df_trades['size_low'] = df_trades['size'].apply(lambda x: int(x.split('–')[0].replace('K', '000').replace('M', '000000')))
df_trades['size_low'] *= -1 * ((df_trades['type'] == ' sell').astype(int) * 2 - 1)
df_trades['size_up'] = df_trades['size'].apply(lambda x: int(x.split('–')[1].replace('K', '000').replace('M', '000000').replace(' ', '')))
df_trades['size_up'] *= -1 * ((df_trades['type'] == ' sell').astype(int) * 2 - 1)
df_trades#.groupby(['politician', 'ticker', 'week_date'])['size_low', 'size_up'].sum()

,politician,trade_issuer,ticker,traded,owner,type,size,price,week_date,size_low,size_up
1,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Undisclosed,sell,15K–50K,78.29,2023-08-11,-15000,-50000
2,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Undisclosed,buy,15K–50K,77.89,2023-08-11,15000,50000
3,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Undisclosed,sell,15K–50K,78.29,2023-08-11,-15000,-50000
4,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Undisclosed,buy,15K–50K,77.89,2023-08-11,15000,50000
5,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Spouse,sell,15K–50K,78.29,2023-08-11,-15000,-50000
...,...,...,...,...,...,...,...,...,...,...,...
40219,Doug Lamborn,NetApp Inc,NTAP,2020-09-08,Spouse,sell,1K–15K,45.2,2020-09-11,-1000,-15000
40220,Doug Lamborn,NetApp Inc,NTAP,2020-09-08,Undisclosed,sell,1K–15K,45.2,2020-09-11,-1000,-15000
40221,Doug Lamborn,NetApp Inc,NTAP,2020-09-08,Spouse,sell,1K–15K,45.2,2020-09-11,-1000,-15000
40222,Doug Lamborn,NetApp Inc,NTAP,2020-09-14,Undisclosed,buy,1K–15K,45.38,2020-09-18,1000,15000


## Compute Capitol Hill long-short portfolio

In [82]:
new_index = pd.MultiIndex.from_product([df_trades['ticker'].unique(), dates], names=['ticker', 'week_date'])

df_trades = df_trades.sort_values(by=['ticker', 'week_date'])

weekly_cumulative = df_trades.groupby(['ticker', 'week_date'])['size_low', 'size_up'].sum().groupby(level=0).cumsum().reindex(new_index)
weekly_cumulative = weekly_cumulative.ffill().fillna(0)
weekly_cumulative['mean_size'] = (weekly_cumulative['size_low'] + weekly_cumulative['size_up']) / 2
weekly_cumulative

/home/lpupp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


size_low  size_up  mean_size
ticker week_date                               
NTAP   2020-09-04         0        0        0.0
       2020-09-11     -3000   -45000   -24000.0
       2020-09-18         0        0        0.0
       2020-09-25         0        0        0.0
       2020-10-02         0        0        0.0
...                     ...      ...        ...
FHB    2023-07-14     -2000   -30000   -16000.0
       2023-07-21     -2000   -30000   -16000.0
       2023-07-28     -2000   -30000   -16000.0
       2023-08-04     -2000   -30000   -16000.0
       2023-08-11     -2000   -30000   -16000.0

[296142 rows x 3 columns]

In [83]:
df_prices = pd.DataFrame()
for ticker in df_trades.ticker.dropna().unique():
    df_ticker_price = pd.read_csv(os.path.join(PATH_DATA_PRICES, f'{ticker}.csv'))
    df_ticker_price['Ticker'] = ticker
    df_prices = df_prices.append(df_ticker_price.loc[:, ['Ticker', 'Date', 'Close']])

df_prices['Date'] = pd.to_datetime(df_prices['Date'])
df_prices['Ticker'] = df_prices['Ticker'].astype('category')

weekly_cumulative = weekly_cumulative.merge(df_prices, how='left', left_index=True, right_on=['Ticker', 'Date'])

del df_prices, df_ticker_price

In [101]:
wealth = 1_000_000
for i, date in enumerate(dates):

    portfolio = weekly_cumulative.loc[(weekly_cumulative.Date == date), ['Ticker', 'Close', 'mean_size']]

    if i != 0:
        last_portfolio = last_portfolio.merge(portfolio.loc[:, ['Ticker', 'Close']], how='left', on='Ticker', suffixes=('', '_new'))
        wealth = (last_portfolio.n_assets / last_portfolio.Close_new).sum()

    print(wealth)

    portfolio['relative_position'] = portfolio['mean_size'] / np.abs(portfolio.mean_size).sum()
    portfolio['value_of_trade'] = portfolio['relative_position'] * portfolio['Close']
    portfolio['n_assets'] = wealth * portfolio['value_of_trade']

    last_portfolio = portfolio.copy()
    


1000000
281422.9488486845
-140025.2100747475
53831.03620610176
-24681.4831193095
13387.856108368522
-6714.227599362912
3090.3887516396835
-1314.3347731648223
456.7564692862687
-166.9217836937457
59.3011678640993
-15.290496105375446
3.7694484368456123
-0.8734547464282527
0.2537892194318377
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


## Compute insider Capitol Hill long-short portfolio
### Load and merge firm's sector/industry information

In [12]:
df_trades = df_trades.merge(pd.read_csv(os.path.join(PATH_DATA, 'YahooFinance_industry.csv')), how='left', on='ticker')
df_trades.dropna(subset=['industry', 'sector'], inplace=True)
df_trades.head(10)

,politician,trade_issuer,ticker,traded,owner,type,size,price,week_date,size_low,size_up,sector_x,industry_x,sector_y,industry_y,sector,industry
0,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Undisclosed,sell,15K–50K,78.29,2023-08-11,-15000,-50000,Technology,Computer Hardware,Technology,Computer Hardware,Technology,Computer Hardware
1,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Undisclosed,buy,15K–50K,77.89,2023-08-11,15000,50000,Technology,Computer Hardware,Technology,Computer Hardware,Technology,Computer Hardware
2,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Undisclosed,sell,15K–50K,78.29,2023-08-11,-15000,-50000,Technology,Computer Hardware,Technology,Computer Hardware,Technology,Computer Hardware
3,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Undisclosed,buy,15K–50K,77.89,2023-08-11,15000,50000,Technology,Computer Hardware,Technology,Computer Hardware,Technology,Computer Hardware
4,Doug Lamborn,NetApp Inc,NTAP,2023-08-07,Spouse,sell,15K–50K,78.29,2023-08-11,-15000,-50000,Technology,Computer Hardware,Technology,Computer Hardware,Technology,Computer Hardware
5,Doug Lamborn,NetApp Inc,NTAP,2023-08-08,Spouse,buy,15K–50K,77.89,2023-08-11,15000,50000,Technology,Computer Hardware,Technology,Computer Hardware,Technology,Computer Hardware
6,Bill Keating,PepsiCo Inc,PEP,2023-08-10,Undisclosed,buy,15K–50K,N/A,2023-08-11,15000,50000,Consumer Defensive,Beverages—Non-Alcoholic,Consumer Defensive,Beverages—Non-Alcoholic,Consumer Defensive,Beverages—Non-Alcoholic
7,Lois Frankel,Fluor Corp,FLR,2023-08-04,Undisclosed,sell,1K–15K,35.94,2023-08-04,-1000,-15000,Industrials,Engineering & Construction,Industrials,Engineering & Construction,Industrials,Engineering & Construction
8,Lois Frankel,Insperity Inc,NSP,2023-08-07,Undisclosed,sell,1K–15K,97.4,2023-08-11,-1000,-15000,Industrials,Staffing & Employment Services,Industrials,Staffing & Employment Services,Industrials,Staffing & Employment Services
9,Lois Frankel,Insperity Inc,NSP,2023-08-04,Undisclosed,sell,1K–15K,94.15,2023-08-04,-1000,-15000,Industrials,Staffing & Employment Services,Industrials,Staffing & Employment Services,Industrials,Staffing & Employment Services


### Load and process committee membership data

In [145]:
with open(os.path.join(PATH_DATA, 'ballotpedia.yml'), 'r') as f_nm:
    committee_membership = yaml.load(f_nm)

committees = [[x for x in v.values()] for v in committee_membership.values()]
committees = flatten_list(flatten_list(committees))

/home/lpupp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


### Manual cleaning of committees

In [129]:
ranks = [
    'ranking member',
    'vice',
    'covice',
    'chair',
    'ex officio',
    'woman',
    'man',
]

hand_filter = [
    'AN ACT to amend 1961 PA 236',
    'Ballotpedia monitors legislation that',
    'Key votes',
    'MI HB4184 - Courts: district',
    'See also: Key votes',
    "The following table lists bills this",
]

word_list = [
    'united states house of representatives select committee on the',
    'joint committee on',
    'joint',
    'senate committee on',
    'senate committee',
    'senate',
    'house committee on',
    'house committee',
    'us house',
    'house ',
    'subcommittee for',
    'subcommittee on',
    'committee on',
    'subcommittee',
    'committee',
    'new york state assembly',
    'oregon state legislature',
    'oklahoma state'
]

for flt in hand_filter:
    committees = [x for x in committees if flt not in x]

committees = [re.sub("[\(\[].*?[\)\]]", '',  x).lower().translate(str.maketrans('', '', string.punctuation)) for x in committees]

for rank in ranks:
    committees = [x.replace(rank, '') for x in committees]

for wrd in word_list:
    committees = [x.replace(wrd, '') for x in committees]

#   Fixes
committees = [x.replace('hu ', 'human') for x in committees]
committees = [x.replace(' agement', ' management') for x in committees]
committees = [x.replace(' ufacturing', ' manufacturing') for x in committees]
committees = [x.replace(' sers', ' services') for x in committees]
committees = [x.replace(' humansers', ' humanservices') for x in committees]
committees = [x.replace('  ', ' ') for x in committees]

committees = [x.strip() for x in committees]
committees = [x for x in committees if len(x) > 1]
committees.remove('')

committees = list(set(committees))


ValueError: list.remove(x): x not in list

# TODO


The next thing we need to do is allocate the committee memberships to each politician-date.
Then we need to compute the similarities between each trade's firm sector-industry and each politician-committee-membership and take the max.

Then, we filter by max-similarity and build a long short portfolio to see if we beat the vanilla portfolio.

Finally, we compute the factor model alphas.

Bam, done.


### compute similarity score between committee and industry
https://medium.com/@gulsum.budakoglu/calculating-sentence-similarity-using-bert-model-8c0e796e25d4


In [133]:
from sentence_transformers import SentenceTransformer # !pip install -U sentence-transformers
from sklearn.metrics.pairwise import cosine_similarity

In [134]:
model = SentenceTransformer("bert-base-uncased")

No sentence-transformers model found with name /home/lpupp/.cache/torch/sentence_transformers/bert-base-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/lpupp/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification

In [141]:
committee_embeddings =  model.encode(committees)
committee_embeddings

array([[ 0.58017975, -0.0829256 , -0.272468  , ..., -0.46707588,
        -0.6274467 ,  0.64819366],
       [-0.07544694,  0.2579933 , -0.46547416, ..., -0.42592263,
         0.01630715, -0.16142674],
       [ 0.58017975, -0.0829256 , -0.272468  , ..., -0.46707588,
        -0.6274467 ,  0.64819366],
       ...,
       [ 0.4789954 , -0.02261051, -0.09868552, ..., -0.18809007,
        -0.71718794,  0.16586068],
       [ 0.39914322,  0.41808328,  0.03568466, ..., -0.5756817 ,
         0.3438176 ,  0.04368131],
       [ 0.45951706,  0.23269358, -0.36356136, ..., -0.2689939 ,
         0.30501485,  0.07746209]], dtype=float32)

In [142]:
# committee_embeddings =  model.encode(committees)
industry_embeddings =  model.encode(list(set([f'{x} {y}' for x,y in zip(df_trades.industry, df_trades.sector)])))
industry_embeddings

array([[-0.30769634, -0.1265342 , -0.10594543, ..., -0.31816056,
        -0.24945472, -0.16355608],
       [ 0.22188446, -0.02389912,  0.30956712, ..., -0.0694536 ,
        -0.1814858 , -0.4653742 ],
       [ 0.41646463,  0.43406352, -0.3573962 , ..., -0.38167375,
         0.13207503, -0.292098  ],
       ...,
       [ 0.09917255,  0.18053189, -0.20906813, ..., -0.12231103,
         0.14577201, -0.10729332],
       [ 0.44171143,  0.15452386, -0.08167715, ..., -0.5185389 ,
        -0.12932463, -0.2723732 ],
       [ 0.3167265 , -0.03553402,  0.38881725, ..., -0.3513982 ,
         0.04491668, -0.20090838]], dtype=float32)

In [144]:
similarities = cosine_similarity(committee_embeddings, industry_embeddings) 
similarities


array([[0.40746894, 0.33026227, 0.44205117, ..., 0.44300905, 0.4287465 ,
        0.2670247 ],
       [0.5149394 , 0.515817  , 0.6801798 , ..., 0.60180044, 0.5927875 ,
        0.46793902],
       [0.40746894, 0.33026227, 0.44205117, ..., 0.44300905, 0.4287465 ,
        0.2670247 ],
       ...,
       [0.3886283 , 0.34482965, 0.41764635, ..., 0.39992267, 0.37742138,
        0.29943243],
       [0.5579487 , 0.5828255 , 0.64227855, ..., 0.7292484 , 0.6107135 ,
        0.5218331 ],
       [0.554019  , 0.5814209 , 0.64868397, ..., 0.76161057, 0.59975773,
        0.52486944]], dtype=float32)